In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [4]:
import numpy as np
import os

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [7]:
data_dir = 'data'

In [8]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}

In [9]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'test']}

In [10]:
dataset_sizes  = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 793, 'test': 207}

In [11]:
class_names = image_datasets['train'].classes
class_names

['cavalo',
 'comida',
 'dinossauro',
 'elefante',
 'flor',
 'indio',
 'montanha',
 'monumento',
 'onibus',
 'praia']

In [12]:
model = models.resnet34(pretrained=True)

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [15]:
def train(model, dataloader, optimizer, criterion, epoch):
    model.train()
    
    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step() 
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.data.item()))

In [16]:
def val(model, dataloader, criterion):
    model.eval()
    
    val_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
            val_loss += criterion(prediction, y).item()
            
            y_hat = prediction.max(1, keepdim=True)[1]
            correct += y_hat.eq(y.view_as(y_hat)).sum().item()
    
    val_loss /= len(dataloader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))

In [17]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model.to(device);

In [20]:
for epoch in range(1, 30+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['test'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.9

Train Epoch: 1 [0/793 (0%)]	Loss: 0.967124
Train Epoch: 1 [640/793 (80%)]	Loss: 1.076179

Test set: Average loss: 0.0237, Accuracy: 155/207 (75%)

Train Epoch: 2 [0/793 (0%)]	Loss: 0.918668
Train Epoch: 2 [640/793 (80%)]	Loss: 0.976166

Test set: Average loss: 0.0212, Accuracy: 162/207 (78%)

Train Epoch: 3 [0/793 (0%)]	Loss: 0.986874
Train Epoch: 3 [640/793 (80%)]	Loss: 0.997521

Test set: Average loss: 0.0318, Accuracy: 140/207 (68%)

Train Epoch: 4 [0/793 (0%)]	Loss: 0.664081
Train Epoch: 4 [640/793 (80%)]	Loss: 0.846328

Test set: Average loss: 0.0218, Accuracy: 167/207 (81%)

Train Epoch: 5 [0/793 (0%)]	Loss: 0.885187
Train Epoch: 5 [640/793 (80%)]	Loss: 0.679791

Test set: Average loss: 0.0199, Accuracy: 167/207 (81%)

Train Epoch: 6 [0/793 (0%)]	Loss: 0.642986
Train Epoch: 6 [640/793 (80%)]	Loss: 0.797770

Test set: Average loss: 0.0205, Accuracy: 167/207 (81%)

Train Epoch: 7 [0/793 (0%)]	Loss: 0.569787
Train Epoch: 7 [640/793 (80%)]	Loss: 1.054065

Test set: Average loss: 0.02

In [24]:
for epoch in range(1, 5+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['test'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.33

Train Epoch: 1 [0/793 (0%)]	Loss: 0.122392
Train Epoch: 1 [640/793 (80%)]	Loss: 0.305244

Test set: Average loss: 0.0074, Accuracy: 191/207 (92%)

Train Epoch: 2 [0/793 (0%)]	Loss: 0.327857
Train Epoch: 2 [640/793 (80%)]	Loss: 0.290686

Test set: Average loss: 0.0072, Accuracy: 190/207 (92%)

Train Epoch: 3 [0/793 (0%)]	Loss: 0.163602
Train Epoch: 3 [640/793 (80%)]	Loss: 0.478039

Test set: Average loss: 0.0074, Accuracy: 191/207 (92%)

Train Epoch: 4 [0/793 (0%)]	Loss: 0.281619
Train Epoch: 4 [640/793 (80%)]	Loss: 0.273790

Test set: Average loss: 0.0077, Accuracy: 191/207 (92%)

Train Epoch: 5 [0/793 (0%)]	Loss: 0.417456
Train Epoch: 5 [640/793 (80%)]	Loss: 0.258530

Test set: Average loss: 0.0074, Accuracy: 190/207 (92%)

